<a href="https://colab.research.google.com/github/katifrahim/Reducing-the-computational-demand-of-3D-CNNs/blob/main/Pruning_3D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=83a12d4ff55866149a6ebe3d9221b0c51f9073c0de3c958bf1fa14c1ff837144
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from torch.utils.data import Dataset, DataLoader
import GPUtil
from tabulate import tabulate
import copy

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Custom dataset and dataloader for the 3D CNN

class VoxelGridDataset(Dataset):
  def __init__(self, root_dir, mode, transform=None):
    self.root_dir = root_dir
    self.mode = mode
    self.transform = transform
    self.file_paths = []
    self.categories = []
    self.category_to_index = {}
    self.index_to_category = {}

    categories = os.listdir(root_dir)
    categories.sort()

    for idx, category in enumerate(categories):
      self.category_to_index[category] = idx
      self.index_to_category[idx] = category

    for category in categories:
      category_path = os.path.join(root_dir, category)
      category_mode_path = os.path.join(category_path, mode)
      for file in os.listdir(category_mode_path):
        self.file_paths.append(os.path.join(category_mode_path, file))
        self.categories.append(category)

  def __len__(self):
    return len(self.file_paths)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    voxel_grid_array = np.load(self.file_paths[idx])['arr_0']
    category = self.categories[idx]
    category_idx = self.category_to_index[category]

    if self.transform:
      voxel_grid_array = self.transform(voxel_grid_array)

    return voxel_grid_array, category_idx

class ToTensor(object):
  def __call__(self, voxel_grid_array):
    return torch.from_numpy(voxel_grid_array).float()

root_dir = '/content/drive/MyDrive/ModelNet10_arrays'

train_dataset = VoxelGridDataset(root_dir, mode='train', transform=ToTensor())
test_dataset = VoxelGridDataset(root_dir, mode='test', transform=ToTensor())

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # shuffle is set true to prevent overfitting
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # shuffle is set false for accurate and consistent evaluation

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)
print(torch.cuda.get_device_name(0))

cuda
Tesla T4


In [ ]:
# Custom 3D CNN
class CNN(nn.Module):
    def __init__(self):
      super(CNN, self).__init__()
      self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
      self.conv1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
      self.conv2 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
      self.relu = nn.ReLU()
      self.flatten = nn.Flatten()
      self.fc1 = nn.Linear(32 * 25 * 25 * 25, 128)
      self.fc2 = nn.Linear(128, 10)  # ModelNet10 Dataset contains 10 different labels
    def forward(self, x):
      # input 1x100x100x100 output 1x50x50x50
      x = self.pool(x)
      # input 1x50x50x50 output 16x50x50x50
      x = self.relu(self.conv1(x))
      # input 16x50x50x50 output 32x50x50x50
      x = self.relu(self.conv2(x))
      # input 32x50x50x50 output 32x25x25x25
      x = self.pool(x)
      # input 32x25x25x25 output 500000
      x = self.flatten(x)
      # input 500000 output 128
      x = self.relu(self.fc1(x))
      # input 128 output 10
      x = self.fc2(x)
      return x

# Instantiating
model = CNN()
model.to(device)

# Training
print("Training...")
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
epochs = 2
model.train()
for epoch in range(epochs):
  samples = 0
  for data, labels in train_dataloader:
    data, labels = data.to(device), labels.to(device)
    data = data.unsqueeze(1)
    predicted_labels = model(data)
    loss = loss_function(predicted_labels, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    samples += len(labels)
    print(f'Progress: Epoch {epoch+1} - {(samples/len(train_dataset))*100} % Done!')
print('Finished training!')

# Testing
print("Testing...")
total_samples = 0
correct_predictions = 0
gpu_utilizations = []
model.eval()
with torch.no_grad():
  for data, labels in test_dataloader:
    data, labels = data.to(device), labels.to(device)
    data = data.unsqueeze(1)
    torch.cuda.synchronize()
    predicted_labels = model(data)
    torch.cuda.synchronize()
    gpu_utilization = GPUtil.getGPUs()[0].load * 100
    gpu_utilizations.append(gpu_utilization)
    predicted_labels = predicted_labels.argmax(dim=1)
    total_samples += len(labels)
    for i in range(len(labels)):
      if predicted_labels[i] == labels[i]:
        correct_predictions += 1
    print(f'Progress: {(total_samples/len(test_dataset))*100} % Done!')
print('Finished testing! \n')

# Results
average_gpu_utilization_batch = sum(gpu_utilizations) / len(gpu_utilizations)
average_gpu_utilization_inference = average_gpu_utilization_batch / batch_size
accuracy = (correct_predictions / total_samples) * 100

print(f"Accuracy: {accuracy}% | GPU utilization: {average_gpu_utilization_inference}%")

Training...
Progress: Epoch 1 - 1.6036081182660988 % Done!
Progress: Epoch 1 - 3.2072162365321977 % Done!
Progress: Epoch 1 - 4.810824354798296 % Done!
Progress: Epoch 1 - 6.414432473064395 % Done!
Progress: Epoch 1 - 8.018040591330495 % Done!
Progress: Epoch 1 - 9.621648709596592 % Done!
Progress: Epoch 1 - 11.22525682786269 % Done!
Progress: Epoch 1 - 12.82886494612879 % Done!
Progress: Epoch 1 - 14.432473064394888 % Done!
Progress: Epoch 1 - 16.03608118266099 % Done!
Progress: Epoch 1 - 17.639689300927085 % Done!
Progress: Epoch 1 - 19.243297419193183 % Done!
Progress: Epoch 1 - 20.846905537459286 % Done!
Progress: Epoch 1 - 22.45051365572538 % Done!
Progress: Epoch 1 - 24.05412177399148 % Done!
Progress: Epoch 1 - 25.65772989225758 % Done!
Progress: Epoch 1 - 27.261338010523676 % Done!
Progress: Epoch 1 - 28.864946128789775 % Done!
Progress: Epoch 1 - 30.468554247055874 % Done!
Progress: Epoch 1 - 32.07216236532198 % Done!
Progress: Epoch 1 - 33.675770483588074 % Done!
Progress: Ep

In [ ]:
# Iterative L1 norm weight pruning

print("L1 norm weight pruning \n")

pruning_epochs = 10
training_epochs = 1
pruning_ratio = 0.0
results_array = []
for pruning_epoch in range(pruning_epochs):

  pruning_model = copy.deepcopy(model)
  pruning_model.to(device)
  pruning_ratio += 0.1

  # Pruning
  for module in pruning_model.modules():
    if isinstance(module, nn.Conv3d):
      prune.l1_unstructured(module, name='weight', amount=pruning_ratio)
      prune.remove(module, "weight")

  # Testing
  total_samples = 0
  correct_predictions = 0
  gpu_utilizations = []
  pruning_model.eval()
  with torch.no_grad():
    for data, labels in test_dataloader:
      data, labels = data.to(device), labels.to(device)
      data = data.unsqueeze(1)
      torch.cuda.synchronize()
      predicted_labels = pruning_model(data)
      torch.cuda.synchronize()
      gpu_utilization = GPUtil.getGPUs()[0].load * 100
      gpu_utilizations.append(gpu_utilization)
      predicted_labels = predicted_labels.argmax(dim=1)
      total_samples += len(labels)
      for i in range(len(labels)):
        if predicted_labels[i] == labels[i]:
          correct_predictions +=1

  # Results
  pruning_percentage = pruning_ratio*100
  accuracy =  correct_predictions/ total_samples * 100
  average_gpu_utilization_batch = sum(gpu_utilizations) / len(gpu_utilizations)
  average_gpu_utilization_inference = average_gpu_utilization_batch / batch_size
  array = [pruning_percentage, accuracy, average_gpu_utilization_inference]
  results_array.append(array)

  # Displaying the results
  print(f"Pruning percentage for each conv layer: {pruning_percentage}% | Accuracy: {accuracy}% | GPU utilization: {average_gpu_utilization_inference}%")
  headers_array = ['Pruning percentage per conv layer (%)', 'Accuracy (%)', 'GPU utilization per inference (%)']
  table = tabulate(results_array, headers_array, tablefmt="grid")
  print(table)

L1 norm weight pruning 

Pruning percentage for each conv layer: 10.0% | Accuracy: 78.9647577092511% | GPU utilization: 0.7770833333333333%
+-----------------------------------------+----------------+-------------------------------------+
|   Pruning percentage per conv layer (%) |   Accuracy (%) |   GPU utilization per inference (%) |
+=========================================+================+=====================================+
|                                      10 |        78.9648 |                            0.777083 |
+-----------------------------------------+----------------+-------------------------------------+
Pruning percentage for each conv layer: 20.0% | Accuracy: 79.07488986784142% | GPU utilization: 0.6083333333333333%
+-----------------------------------------+----------------+-------------------------------------+
|   Pruning percentage per conv layer (%) |   Accuracy (%) |   GPU utilization per inference (%) |
+=========================================+========

In [ ]:
# Iterative L1 norm filter pruning

print("L1 norm filter pruning \n")

pruning_epochs = 10
training_epochs = 1
pruning_ratio = 0.0
results_array = []
for pruning_epoch in range(pruning_epochs):

  pruning_model = copy.deepcopy(model)
  pruning_model.to(device)
  pruning_ratio += 0.1

  # Pruning
  for module in pruning_model.modules():
    if isinstance(module, nn.Conv3d):
      prune.ln_structured(module, name='weight', amount=pruning_ratio, n=1, dim=0)
      prune.remove(module, "weight")

  # Testing
  total_samples = 0
  correct_predictions = 0
  gpu_utilizations = []
  pruning_model.eval()
  with torch.no_grad():
    for data, labels in test_dataloader:
      data, labels = data.to(device), labels.to(device)
      data = data.unsqueeze(1)
      torch.cuda.synchronize()
      predicted_labels = pruning_model(data)
      torch.cuda.synchronize()
      gpu_utilization = GPUtil.getGPUs()[0].load * 100
      gpu_utilizations.append(gpu_utilization)
      predicted_labels = predicted_labels.argmax(dim=1)
      total_samples += len(labels)
      for i in range(len(labels)):
        if predicted_labels[i] == labels[i]:
          correct_predictions +=1

  # Results
  pruning_percentage = pruning_ratio*100
  accuracy =  correct_predictions/ total_samples * 100
  average_gpu_utilization_batch = sum(gpu_utilizations) / len(gpu_utilizations)
  average_gpu_utilization_inference = average_gpu_utilization_batch / batch_size
  array = [pruning_percentage, accuracy, average_gpu_utilization_inference]
  results_array.append(array)

  # Displaying the results
  print(f"Pruning percentage for each conv layer: {pruning_percentage}% | Accuracy: {accuracy}% | GPU utilization: {average_gpu_utilization_inference}%")
  headers_array = ['Pruning percentage per conv layer (%)', 'Accuracy (%)', 'GPU utilization per inference (%)']
  table = tabulate(results_array, headers_array, tablefmt="grid")
  print(table)